In [1]:
class Configuration:
    DATASET_SIZE = 4000
    BATCH_SIZE = 64
    EPOCHS = 40
    MAX_FEATURES = 100
    EMBEDDING_DIM = 256
    HIDDEN_DIM = 300
    MAX_SEQ_LEN = 50
    TEST_SIZE = 0.2
    VAL_SIZE = 0.2

In [2]:
import re

texts_en = []
texts_pl = []

with open('en-pl.txt') as file:
    for i, line in enumerate(file):
        if i>=Configuration.DATASET_SIZE:
            break
        en, pl = tuple(line.split('\t'))
        re.sub(r'[\t\n]', '', pl)

        texts_pl.append('\t ' + pl.strip() + ' \n')
        texts_en.append(en.strip())


len(texts_en), len(texts_pl)

(4000, 4000)

In [3]:
from sklearn.model_selection import train_test_split

en_train_data, en_test_data, pl_train_data, pl_test_data = train_test_split(
    texts_en, 
    texts_pl,
    test_size=Configuration.TEST_SIZE,
    random_state=42
)
en_train_data, en_val_data, pl_train_data, pl_val_data = train_test_split(
    en_train_data,
    pl_train_data,
    test_size=Configuration.TEST_SIZE,
    random_state=42
)
print(en_test_data[0])
print(pl_test_data[0])
len(en_train_data), len(en_val_data), len(en_test_data)

Score big bonuses by matching 3 in a row, and being fast.Controls: Click on the hats with the mouse to uncover the pumpkin underneath.Find the identical pairs of pumpkins Rating: 4, Votes: 2, Played 8951 times , Size: (887.451 kb)concentration games, funny games, halloween games, pumpkin games, memory games, brain training games, swapping games, hat games, spooky games, Play free online games, computer flash games.
	 Wynik duże premie, dopasowując 3 w rzędzie, i jest szybki.kontroli Instrukcje: kliknij na kapelusze z myszką, aby odkryć dyni pod spodem. znaleźć identyczne pary dyni , Size: popularność: 4, głosów: 2, Grano 8966 Rrazy (887.451 kb)Stężenie gry, zabawny gry, halloween gry, dynia gry, pamięci gry, szkolenia mózgu gry, zamiana gry, kapelusz gry, Spooky gry, grać bezpłatny online Flash gry szt. gry 2009-xx, Free Online PC Games .NET , Privacy, Contact us 



(2560, 640, 800)

In [4]:
import keras
from keras.preprocessing.text import Tokenizer
import numpy as np

tokenizer_en = Tokenizer(
    num_words=Configuration.MAX_FEATURES,
)
tokenizer_pl = Tokenizer(
    num_words=Configuration.MAX_FEATURES,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~'
)

tokenizer_en.fit_on_texts(texts_en)
tokenizer_pl.fit_on_texts(texts_pl)

en_word_indices = tokenizer_en.word_index
pl_word_indices = tokenizer_pl.word_index
len(en_word_indices), len(pl_word_indices)

(13178, 18255)

In [5]:
pl_word_indices.get('\t'), pl_word_indices.get('\n')

(2, 3)

In [6]:
en_train_data = tokenizer_en.texts_to_sequences(en_train_data)
en_val_data = tokenizer_en.texts_to_sequences(en_val_data)
pl_train_data = tokenizer_pl.texts_to_sequences(pl_train_data)
pl_val_data = tokenizer_pl.texts_to_sequences(pl_val_data)

In [7]:
from keras.preprocessing.sequence import pad_sequences

en_train_data = pad_sequences(
    sequences=en_train_data,
    maxlen=Configuration.MAX_SEQ_LEN,
    padding='post',
    truncating='post'
)

en_val_data = pad_sequences(
    sequences=en_val_data,
    maxlen=Configuration.MAX_SEQ_LEN,
    padding='post',
    truncating='post'
)

pl_train_data = pad_sequences(
    sequences=pl_train_data,
    maxlen=Configuration.MAX_SEQ_LEN,
    padding='post',
    truncating='post'
)

pl_val_data = pad_sequences(
    sequences=pl_val_data,
    maxlen=Configuration.MAX_SEQ_LEN,
    padding='post',
    truncating='post'
)



In [8]:
# building an encoder

from keras import Input, Model
from keras.layers import Embedding, LSTM

encoder_input = Input(shape=(None, ), dtype='int32')
encoder_embedding = Embedding(
    input_dim = Configuration.MAX_FEATURES + 1,
    output_dim = Configuration.EMBEDDING_DIM,
    # input_length = Configuration.MAX_SEQ_LEN - 1
)
encoder_lstm = LSTM(
    Configuration.HIDDEN_DIM,
    return_state = True
)
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding(encoder_input))
encoder = Model(
    inputs=encoder_input, 
    outputs=[encoder_state_h, encoder_state_c]
)

In [9]:
#building a decoder
from keras.layers import TimeDistributed, Dense

decoder_input = Input(shape=(None, ), dtype='int32')
state_h_input = Input(shape=(Configuration.HIDDEN_DIM, ), dtype='float32')
state_c_input = Input(shape=(Configuration.HIDDEN_DIM, ), dtype='float32')
decoder_embedding = Embedding(
    input_dim = Configuration.MAX_FEATURES + 1,
    output_dim = Configuration.EMBEDDING_DIM,
    # input_length = Configuration.MAX_SEQ_LEN
)
decoder_lstm = LSTM(
    Configuration.HIDDEN_DIM,
    return_sequences=True,
    return_state=True,
)
decoder_fc = TimeDistributed(Dense(Configuration.MAX_FEATURES, activation='softmax'))
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(
    decoder_embedding(decoder_input),
    initial_state=[
        state_h_input,
        state_c_input
    ]
)
decoder_output = decoder_fc(decoder_outputs)
decoder = Model(
    inputs=[
        decoder_input,
        state_h_input,
        state_c_input
    ],
    outputs=[
        decoder_output,
        decoder_state_h,
        decoder_state_c
    ]
)

In [10]:
#encoder-decoder model
_outputs, _, _ = decoder_lstm(
    decoder_embedding(decoder_input),
    initial_state=[
        encoder_state_h,
        encoder_state_c
    ]
)
_prob_outputs = decoder_fc(_outputs)
encoder_decoder = Model(
    inputs=[
        encoder_input,
        decoder_input
    ],
    outputs=_prob_outputs
)
encoder_decoder.compile(
    optimizer='adam',
    loss='categorical_crossentropy'
)

In [11]:
# target sequence take the shifted words
import numpy as np
pl_train_data_target = pl_train_data[:,1:]
pl_val_data_target = pl_val_data[:,1:]
# remove the last word from original input sequence to align with target sequence
pl_train_data = pl_train_data[:,:-1]
pl_val_data = pl_val_data[:, :-1]

# one-hot vectorization
# one_hot_train_target = tokenizer_pl.sequences_to_matrix(pl_train_data_target, mode='binary')
# one_hot_val_target = tokenizer_pl.sequences_to_matrix(pl_val_data_target, mode='binary')

def one_hot_vectorize(data, seq_len, dict_size):
    one_hot = np.zeros((len(data), seq_len, dict_size))
    for i, seq in enumerate(data):
        for j, word_idx in enumerate(seq):
            one_hot[i,j,word_idx]=1
    return one_hot

one_hot_train_target = one_hot_vectorize(
    pl_train_data_target,
    Configuration.MAX_SEQ_LEN - 1,
    Configuration.MAX_FEATURES
)
one_hot_val_target = one_hot_vectorize(
    pl_val_data_target,
    Configuration.MAX_SEQ_LEN - 1,
    Configuration.MAX_FEATURES
)
pl_train_data.shape, pl_val_data.shape

((2560, 49), (640, 49))

In [ ]:

encoder_decoder.fit(
    x=[en_train_data, pl_train_data], 
    y=one_hot_train_target,
    batch_size=Configuration.BATCH_SIZE,
    epochs=Configuration.EPOCHS, 
    validation_data=(
        [en_val_data, pl_val_data],
        one_hot_val_target
    ),
    validation_batch_size=Configuration.BATCH_SIZE
)

Epoch 1/100
40/40 [==============================] - 13s 323ms/step - loss: 2.3607 - val_loss: 1.9524
Epoch 2/100
40/40 [==============================] - 14s 342ms/step - loss: 1.7247 - val_loss: 1.5479
Epoch 3/100
40/40 [==============================] - 13s 334ms/step - loss: 1.3567 - val_loss: 1.1994
Epoch 4/100
40/40 [==============================] - 14s 357ms/step - loss: 1.0421 - val_loss: 0.9108
Epoch 5/100
40/40 [==============================] - 13s 314ms/step - loss: 0.8207 - val_loss: 0.7412
Epoch 6/100
40/40 [==============================] - 13s 318ms/step - loss: 0.6867 - val_loss: 0.6415
Epoch 7/100
40/40 [==============================] - 13s 318ms/step - loss: 0.5996 - val_loss: 0.5641
Epoch 8/100
40/40 [==============================] - 13s 319ms/step - loss: 0.5373 - val_loss: 0.5142
Epoch 9/100
40/40 [==============================] - 13s 332ms/step - loss: 0.4932 - val_loss: 0.4856
Epoch 10/100
40/40 [==============================] - 14s 350ms/step - loss: 0.477

In [35]:
def initiate_state(sentence):
    en_idx_word = tokenizer_en.index_word
    en_word_idx = tokenizer_en.word_index
    pl_idx_word = tokenizer_pl.index_word
    pl_word_idx = tokenizer_pl.word_index
    start_token_idx = pl_word_idx.get('\t', None)
    if not start_token_idx:
        raise ValueError('there must be a start token')
    
    sequence = tokenizer_en.texts_to_sequences([sentence])
    sequence = pad_sequences( 
        sequences=sequence,
        maxlen=Configuration.MAX_SEQ_LEN,
        padding='post',
        truncating='post'
    )
    encoder_states = encoder.predict(sequence)
    return ([start_token_idx], ['\t'], [1.0]), encoder_states

In [36]:
def update_state(token_triple, states):
    en_idx_word = tokenizer_en.index_word
    en_word_idx = tokenizer_en.word_index
    pl_idx_word = tokenizer_pl.index_word
    pl_word_idx = tokenizer_pl.word_index
    
    indices, tokens, probabs = token_triple
    probs, state_h, state_c = decoder.predict([np.array([[indices[-1]]]), states[0], states[1]])
    states_new = [state_h, state_c]
    probs = probs[0, -1, :]
    index_new = np.argmax(probs)
    token_new = pl_idx_word.get(index_new, None)
    prob_new = np.max(probs)
    token_triple_new = (
        indices + [index_new],
        tokens + [token_new],
        probabs + [prob_new]
    )
    return token_triple_new, states_new

In [39]:
def infer_lstm(sentence):
    token_triple, states = initiate_state(sentence)
    for _ in range(Configuration.MAX_SEQ_LEN):
        token_triple, states = update_state(token_triple, states)
        if token_triple[1][-1] == '\n' or token_triple[1][-1] is None:
            token_triple[1][-1] = ''
            break
    return ' '.join(token_triple[1])


In [40]:
for i, (en_sentence, pl_sentence) in enumerate(zip(en_test_data, pl_test_data)):
    translation = infer_lstm(en_sentence)
    print(f'Original sentence: \n\t {en_sentence}')
    print(f'Original translation: \n\t {pl_sentence}')
    print(f'Seq2Seq translation: \n\t {translation}')
    if i==2:
        break
        

Original sentence: 
	 Friday, July 15, 2016 in Everett the weather will be like this: during the night the air temperature drops to +12...+14°C, dew point: +11,8°C; ratio of temperature, wind speed and humidity: Very comfortable; precipitation is not expected, light breeze wind blowing from the north-west at a speed of 7-11 km/h , clear sky; day length is 15:38
Original translation: 
	 	 W piątek, 15 lipca 2016 w Everett charakter pogoda będzie taka: w nocy temperatura powietrza spada do +12...+14°C, punkt rosy: +11,8°C; wartości temperatury i wilgotności: bardzo dobrze; atmosferyczne nie ma, wiatr Słaby wiatr, podmuchowy z od północnego-zachodu z prędkością 2-3 m/s , niebo czyste; długość dnia 15:38 

Seq2Seq translation: 
	 	 wpisz dane a my oddzwonimy do ciebie imię i nazwisko telefon zadaj pytanie imię i nazwisko email telefon pytanie oferta numer na sprzedaż dodano 2009 województwo kujawsko pomorskie 
Original sentence: 
	 See more What's New in Version 1.2.2(07/21/2017) Now Cart 